In [2]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.datasets import mnist

import visualkeras

In [3]:
# loading dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X = np.concatenate([X_train, X_test], axis = 0)
y = np.concatenate([y_train, y_test], axis = 0)

# reshapring 
X = X.reshape(X.shape[0], 28, 28, 1) 

# casting to float 
X.astype('float32')

# normalizing 
X = X / 255

# encoding output values
y = to_categorical(y)

In [9]:
INPUT_SHAPE = (28,28,1)
OUTPUT_SHAPE = 10
BATCH_SIZE = 6
EPOCHS = 2 
VERBOSE = 2
K_FOLDS = 10

In [10]:
def create_model():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=INPUT_SHAPE))
    model.add(MaxPool2D((2,2)))

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPool2D((2,2)))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    return model

In [11]:
model = create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [12]:
scores = []

for i in range(K_FOLDS):
    print("Training on Fold: ",i+1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3
                                               ,random_state = np.random.randint(1,1000, 1)[0])
    
    model = create_model()
    model.fit(
        X_train, 
        y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=VERBOSE,
        validation_split=0.3
    )
    
    _, score = model.evaluate(X_test, y_test, verbose = 0)
    scores.append(score)
   
    print("======="*12, end="\n\n\n")

Training on Fold:  1
Epoch 1/2
5717/5717 - 30s - loss: 0.1961 - accuracy: 0.9414 - val_loss: 0.0692 - val_accuracy: 0.9789 - 30s/epoch - 5ms/step
Epoch 2/2
5717/5717 - 27s - loss: 0.0763 - accuracy: 0.9789 - val_loss: 0.0606 - val_accuracy: 0.9826 - 27s/epoch - 5ms/step


Training on Fold:  2
Epoch 1/2
5717/5717 - 26s - loss: 0.2041 - accuracy: 0.9387 - val_loss: 0.0688 - val_accuracy: 0.9814 - 26s/epoch - 5ms/step
Epoch 2/2
5717/5717 - 25s - loss: 0.0721 - accuracy: 0.9791 - val_loss: 0.0529 - val_accuracy: 0.9842 - 25s/epoch - 4ms/step


Training on Fold:  3
Epoch 1/2
5717/5717 - 26s - loss: 0.2040 - accuracy: 0.9377 - val_loss: 0.0520 - val_accuracy: 0.9845 - 26s/epoch - 5ms/step
Epoch 2/2
5717/5717 - 58s - loss: 0.0739 - accuracy: 0.9790 - val_loss: 0.0483 - val_accuracy: 0.9865 - 58s/epoch - 10ms/step


Training on Fold:  4
Epoch 1/2
5717/5717 - 39s - loss: 0.2005 - accuracy: 0.9395 - val_loss: 0.0594 - val_accuracy: 0.9831 - 39s/epoch - 7ms/step
Epoch 2/2
5717/5717 - 42s - loss: 

In [13]:

print(f"K{K_FOLDS} Fold cross results:")
print("Accuracy:", np.mean(scores) * 100)
print("Std:",      np.std(scores) * 100)
print("K value:", len(scores))

K10 Fold cross results:
Accuracy: 52.071143463253975
Std: 46.27433199542827
K value: 10


In [7]:
# visualkeras.layered_view(models[0], legend=True, draw_volume=False,spacing=30)

In [15]:
scores = [score for _, score in scores]

In [16]:
scores

[0.9822381138801575,
 0.9834761619567871,
 0.9855238199234009,
 0.9853333234786987,
 0.979190468788147,
 0.9850000143051147,
 0.984000027179718,
 0.9837619066238403,
 0.984333336353302,
 0.9811905026435852]